In [1]:
import pandas as pd
import numpy as np
from significance_of_mean_cuda import significance_of_mean_cuda
import time
from utils import MW, ttests, qvalues
import sklearn

In [2]:
%load_ext cython
%load_ext rpy2.ipython

/home/ekvall/anaconda3/envs/my-r-env/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/home/ekvall/anaconda3/envs/my-r-env/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [3]:
%%R

require(ggplot2)
install.packages("devtools")
library("devtools")
install_github("bdsegal/fastPerm")

library(fastPerm)

R[write to console]: Loading required package: ggplot2

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cloud.r-project.org/src/contrib/devtools_2.2.2.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 375464 bytes (366 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to cons

In [4]:
!ls "./data/nature18003-s2/"

CPTAC_BC_SupplementaryTable01.xlsx  CPTAC_BC_SupplementaryTable03.xlsx


In [5]:
path = "./data/nature18003-s2/"

In [6]:
T1 = pd.read_excel(path + "CPTAC_BC_SupplementaryTable01.xlsx")
T3 = pd.read_excel(path + "CPTAC_BC_SupplementaryTable03.xlsx")

## Extract important column headings

In [7]:
paiteint_id = T1.columns[0]
stats_col = T1.columns[5: 5 + 3]

### Patient column  

In [8]:
paiteint_id

'Sample ID'

### Cancer phenotype coulmns

In [9]:
stats_col

Index(['ER Status', 'PR Status', 'HER2 Status'], dtype='object')

### Check viable phenotype status

In [10]:
all_lab = list()
for r in T1[stats_col].iterrows():
    all_lab += list(r[1].values)

In [11]:
list(set(all_lab))

['Negative', 'Equivocal', 'Positive', 'Indeterminate']

### Let's drop patient with "Equivocal" and "Indeterminate" status

In [12]:
drop_label = ['Indeterminate', 'Equivocal']

In [13]:
T1.head()

,Sample ID,TCGA ID,Biospecimen Barcode Side,UIUD,PAM50,ER Status,PR Status,HER2 Status,QC Status,Proteome Cluster (see Fig. 3b),...,iTRAQReporterIon,CommonControl,GATA3 Mutation,PIK3CA Mutation,TP53 Mutation,PIK3CA missense mutation in helical domain all tumors,PIK3CA missense mutation in kinase domain all tumors,TP53 Nonsense/Frameshift all tumors,TP53 Missense mutation all tumors,TP53 missense mutations in DNA binding domain all tumors
0,A2-A0CM,TCGA-A2-A0CM,TCGA-A2-A0CM-01A-11-A21V-30,330F7598-824C-4CD6-9303-A27FE74A6695,Basal,Negative,Negative,Negative,pass,1.0,...,116,YES,NaN,NaN,Frame_Shift_Del|Somatic|p.E204fs,0.0,0.0,1.0,NaN,NaN
1,A2-A0D2,TCGA-A2-A0D2,TCGA-A2-A0D2-01A-12-A21W-30,308CCD79-C164-4397-92BC-A1CD243C8E7D,Basal,Negative,Negative,Negative,pass,1.0,...,114,NO,NaN,NaN,Frame_Shift_Del|Somatic|p.P318fs,0.0,0.0,1.0,NaN,NaN
2,A2-A0EQ,TCGA-A2-A0EQ,TCGA-A2-A0EQ-01A-41-A21W-30,04217F65-2907-478B-B0C5-EB65370198DA,Her2,Negative,Negative,Positive,pass,1.0,...,116,NO,NaN,Missense_Mutation|Somatic|p.H1047R,In_Frame_Del|Somatic|p.IY162in_frame_delN,NaN,1.0,NaN,NaN,NaN
3,A2-A0EV,TCGA-A2-A0EV,TCGA-A2-A0EV-01A-41-A21V-30,38F98E5F-7FA4-4C89-8D81-516CF865BEEB,LumA,Positive,Positive,Negative,pass,3.0,...,114,NO,NaN,In_Frame_Ins|Somatic|p.E469in_frame_insDK,NaN,NaN,NaN,0.0,0.0,0.0
4,A2-A0EX,TCGA-A2-A0EX,TCGA-A2-A0EX-01A-41-A21V-30,78271500-147B-40C5-B789-0CF0C7CDBCE2,LumA,Positive,Positive,Negative,pass,3.0,...,116,NO,NaN,Missense_Mutation|Somatic|p.E545K,NaN,1.0,NaN,0.0,0.0,0.0


In [14]:
mask1 = [True if (len(set(r[1].values) & set(drop_label)) == 0) else False for r in T1[stats_col].iterrows()]
T1 = T1[mask1]
T1.reset_index(drop=True, inplace=True)

### Let's divide the dataframe into two dataframes with triple negative (TN) and non-triple negative (NTN)

In [15]:
mask2 = [all([True if v=='Negative' else False for v in r[1].values] ) for r in T1[stats_col].iterrows()]

T1_TNP = T1[np.array(mask2)]
T1_TNP.reset_index(drop=True, inplace=True)

T1_not_TNP = T1[~np.array(mask2)]
T1_not_TNP.reset_index(drop=True, inplace=True)



### Obtain patient ids for NT and NTN

In [16]:
TNPpateintIds = T1_TNP[paiteint_id].values
NotTNPpateintIds = T1_not_TNP[paiteint_id].values

### Remove genes (rows) containing NaN for both TN and NTN i.e., make sure all patients have same the gene when comparing.

In [17]:
def getpatientId(ids):
    patitentList = list()
    for i in ids:
        p_id = T3[T3.columns[[i in c for c in T3.columns]]].columns
        patitentList += list(p_id)
    return patitentList

In [18]:
only_patientDf = T3[getpatientId(NotTNPpateintIds) + getpatientId(TNPpateintIds)]

In [19]:
only_patientDf.dropna(inplace=True)

/home/ekvall/anaconda3/envs/my-r-env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [20]:
NotTNP_df =  only_patientDf[getpatientId(NotTNPpateintIds)]
TNP_df =  only_patientDf[getpatientId(TNPpateintIds)]

In [21]:
NotTNP_Arr, TNP_Arr =NotTNP_df.values, TNP_df.values

In [22]:
NotTNP_Arr.shape

(8051, 80)

In [23]:
TNP_Arr.shape

(8051, 26)

### There are 80 patients without triple negative (NTN) and 26 patients with triple negtive (TN), and with a total of 8051 quantified genes.

# Experiment time

## Let's start with a rather small binsize=6.

In [25]:
start = time.time()

SGM = significance_of_mean_cuda(16, dtype_v=np.uint32,dtype_A=np.float64)
SGM.run(TNP_Arr, NotTNP_Arr)
P_e_list = list(1 - SGM.p_values)

end = time.time()
print(end - start)

13.415400981903076


In [27]:
x = pd.DataFrame(TNP_Arr, columns=list(range(TNP_Arr.shape[1]))).T
y = pd.DataFrame(NotTNP_Arr, columns=list(range(NotTNP_Arr.shape[1]))).T
n_samples = x.shape[1]
outputDf = pd.DataFrame(n_samples * [0], columns=["output"]).T

In [28]:
%R -i x
%R -i y
%R -i outputDf
%R -i n_samples

In [29]:
%%R
runFastPerm <- function(x,y,output, n_samples) {
    e <- list(mode="vector",length=n_samples)
        for (i in 1:n_samples) {
      
            mStopDiffMean(x[[i]], y[[i]])
            valX = fastPerm(x[[i]], y[[i]], testStat = diffMean)
            
            valX <-unlist(valX)

      
            output[[i]] <- as.numeric(valX[1])
      
            }
    

    return(output)


}


In [ ]:
start = time.time()
%R out <- runFastPerm(x,y, outputDf, n_samples)
end = time.time()
print(end - start)

In [ ]:
%R -o out

### Parallelized exact test takes ~3s

In [ ]:
start = time.time()

P_mw_list = MW(TNP_Arr, NotTNP_Arr)

end = time.time()
print(end - start)

In [ ]:
P_mw_list

### Mann-Whitney exact test takes ~1s

In [ ]:
start = time.time()

P_t_list = ttests(TNP_Arr, NotTNP_Arr)

end = time.time()
print(end - start)

In [ ]:
df_e = pd.DataFrame({"p":P_e_list})
df_mw = pd.DataFrame({"p":P_mw_list})
df_tt = pd.DataFrame({"p":P_t_list})

In [ ]:
df_e = qvalues(df_e, pi0=None)
df_mw = qvalues(df_mw, pi0=None)
df_tt = qvalues(df_tt, pi0=None)

In [ ]:
df_e["q"][df_e["q"]<0.05].shape


In [ ]:
df_mw["q"][df_mw["q"]<0.05].shape

In [ ]:
df_tt["q"][df_tt["q"]<0.05].shape

In [ ]:
data_e = df_e["q"][df_e["q"]<0.1]
data_mw = df_mw["q"][df_mw["q"]<0.1]
data_tt = df_tt["q"][df_tt["q"]<0.1]


In [ ]:
import matplotlib.pyplot as plt


In [ ]:

values, base = np.histogram(data_e, bins=40)
#evaluate the cumulative
cumulative = np.cumsum(values)
# plot the cumulative function
plt.plot(base[:-1], cumulative, c='yellow')

values, base = np.histogram(data_mw, bins=40)
#evaluate the cumulative
cumulative = np.cumsum(values)
# plot the cumulative function
plt.plot(base[:-1], cumulative, c='red')

values, base = np.histogram(data_tt, bins=40)
#evaluate the cumulative
cumulative = np.cumsum(values)
# plot the cumulative function
plt.plot(base[:-1], cumulative, c='green')

In [ ]:
def getFDR(df, alpha=0.05):
    dfNew = qvalues(df)
    n_falsePostive = ((dfNew["p"]<=alpha).values !=(dfNew["q"]<=alpha).values).sum()
    FDR = n_falsePostive / dfNew.shape[0]
    return FDR

# Check FDR for $\alpha = 0.05$

In [ ]:
round(getFDR(df_e, alpha=0.05) *100, 1)

In [ ]:
round(getFDR(df_mw, alpha=0.05) * 100, 1)

In [ ]:
round(getFDR(df_tt, alpha=0.05) * 100, 1)

# Check FDR for $\alpha = 0.01$

In [ ]:
round(getFDR(df_e, alpha=0.01) * 100,1)

In [ ]:
round(getFDR(df_mw, alpha=0.01) * 100, 1)

In [ ]:
round(getFDR(df_tt, alpha=0.01) * 100, 1)

# Exact test with bin size 16

In [ ]:
start = time.time()

SGM = significance_of_mean_cuda(16, dtype_v=np.uint32,dtype_A=np.float64)
SGM.run(TNP_Arr, NotTNP_Arr)
P_e_list = list(1 - SGM.p_values)

end = time.time()
print(end - start)

In [ ]:
df_e = pd.DataFrame({"p":P_e_list})


# Check FDR for $\alpha = 0.05$ 

In [ ]:
round(getFDR(df_e, alpha=0.05) * 100, 1)

# Check FDR for $\alpha = 0.01$

In [ ]:
round(getFDR(df_e, alpha=0.01) * 100, 1)